In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-0.6B' ,trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
len(tokenizer)

151669

In [4]:
import torch

(torch.arange(0, 10, 2) / 10).dtype

torch.float32

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import modeling_tinyllm

model_id = 'outputs/buddygpt-qwen3/checkpoint-100'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
model.to('cuda')
prompt = "中国首都是哪?"
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(model.device)
outputs = model.generate(input_ids, max_length=50)
tokenizer.decode(outputs[0], skip_special_tokens=True)